In [2]:
from string import punctuation
import os
import codecs
import numpy as np
from collections import defaultdict


def remove_punctuation(text):
    if(type(text) == float):
        return text
    ans = ""
    for i in text:
        if i not in punctuation and i != '–':
            ans += i
    return ans

In [3]:
def TF(word : str, text):
    return text.count(word) / len(text)
def IDF(word : str, texts):
    texts_with_word_number = 0
    for text in texts:
        texts_with_word_number += 1 if text[0].count(word) > 0 else 0
    return np.log(len(texts) / texts_with_word_number)
def TF_IDF(word : str, text, texts):
    return TF(word, text) * IDF(word, texts)

In [4]:
texts = []
for filename in os.listdir(os.path.join(os.getcwd(), 'TextsLemmatized')):
    file = codecs.open(os.path.join(os.getcwd(), 'TextsLemmatized/' + filename), encoding='utf-8')
    texts.append([file.read()])
    file.close()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Sibdroid\\Desktop\\6sem\\TBIP\\TF-IDF-Clusterization\\TextsLemmatized'

In [482]:
texts = np.array(texts)
for text in texts:
    text = remove_punctuation(text)
texts = np.char.split(texts)

In [571]:
TD_IDF_decades = []
for text in texts:
    text_TD_IDFs = defaultdict(int);
#     print(np.unique(text)[0])
    for word in np.unique(text)[0]:
        text_TD_IDFs[word] = TF_IDF(word, text[0], texts)
    TD_IDF_decades.append(sorted(text_TD_IDFs.items(), key=lambda k_v: k_v[1], reverse=True)[:10])
TD_IDF_decades

[[('сопоставимость', 0.02957448743295105),
  ('параллельный', 0.025101325239797894),
  ('корпус', 0.022892934403814708),
  ('сопоставимый', 0.020671679609245326),
  ('термин', 0.016899707104543457),
  ('сегмент', 0.016899707104543457),
  ('текст', 0.01640265764386209),
  ('comparable', 0.01056231694033966),
  ('ясц', 0.01056231694033966),
  ('перевод', 0.00885929126110514)],
 [('атака', 0.02411291399668829),
  ('прогнозирование', 0.015945880145388128),
  ('аномальный', 0.012756704116310504),
  ('сервис', 0.012756704116310504),
  ('тренд', 0.012756704116310504),
  ('сезонный', 0.012756704116310504),
  ('программа', 0.012691007366678048),
  ('транспортный', 0.009567528087232878),
  ('система', 0.009197691286645262),
  ('исходный', 0.008883705156674634)],
 [('союз', 0.07054728795556225),
  ('будто', 0.037233290865435635),
  ('точно', 0.03287362544546247),
  ('сравнительный', 0.027435056427163103),
  ('словно', 0.023515762651854084),
  ('двусложный', 0.02155611576419958),
  ('русский', 0.0

In [486]:
TD_IDF_fifties = []
for text in texts:
    text_TD_IDFs = defaultdict(int);
    for word in np.unique(text)[0]:
        text_TD_IDFs[word] = TF_IDF(word, text[0], texts)
    TD_IDF_fifties.append(sorted(text_TD_IDFs.items(), key=lambda k_v: k_v[1], reverse=True)[:50])
    
global_TD_IDF_collection = []
for lems in TD_IDF_fifties:
    global_TD_IDF_collection += [x[0] for x in lems]
print(len(global_TD_IDF_collection))
global_TD_IDF_collection = set(global_TD_IDF_collection)
print(len(global_TD_IDF_collection))
# global_TD_IDF_collection

500
454


In [487]:
texts_vectorized = []
for text in texts:
    text_vector = [(1 if word in text[0] else 0) for word in global_TD_IDF_collection]
    texts_vectorized.append(text_vector)
# texts_vectorized

In [488]:
distance_matrix = []
for tv in texts_vectorized:
    d_vec = [np.linalg.norm(np.subtract(tv, x)) for x in texts_vectorized]
    distance_matrix.append(d_vec)
distance_matrix = np.array(distance_matrix)
distance_matrix

array([[ 0.        , 11.74734012, 10.77032961, 11.74734012, 11.26942767,
        11.18033989, 11.09053651, 11.44552314, 10.95445115, 12.16552506],
       [11.74734012,  0.        , 12.        , 10.86278049, 11.70469991,
        10.24695077, 12.04159458, 11.        , 11.66190379, 10.48808848],
       [10.77032961, 12.        ,  0.        , 11.74734012, 10.24695077,
        11.35781669, 11.18033989, 11.44552314, 10.77032961, 12.489996  ],
       [11.74734012, 10.86278049, 11.74734012,  0.        , 11.44552314,
        10.72380529, 11.87434209, 10.90871211, 11.40175425, 11.3137085 ],
       [11.26942767, 11.70469991, 10.24695077, 11.44552314,  0.        ,
        11.5758369 , 11.13552873, 11.74734012, 10.53565375, 12.12435565],
       [11.18033989, 10.24695077, 11.35781669, 10.72380529, 11.5758369 ,
         0.        , 11.48912529, 10.77032961, 10.90871211, 11.18033989],
       [11.09053651, 12.04159458, 11.18033989, 11.87434209, 11.13552873,
        11.48912529,  0.        , 11.40175425

In [576]:
def update_distance_matrix(x : int, y : int, dmatrix):
    dist_matrix = np.delete(dmatrix, x, 1)
    y_d = y - 1 if y > x else y
    dist_matrix = np.delete(dist_matrix, y_d, 1)
    
    old_distance_vec_X = dist_matrix[x]
    old_distance_vec_Y = dist_matrix[y]
    
    dist_matrix = np.delete(dist_matrix, x, 0)
    dist_matrix = np.delete(dist_matrix, y_d , 0)
    
    new_distance_vec = np.add(old_distance_vec_X, old_distance_vec_Y) / 2
    dist_matrix = np.append(dist_matrix, [new_distance_vec], axis=0)
    
    new_distance_vec = np.append(new_distance_vec, 0.0)
    return np.append(dist_matrix, [[k] for k in new_distance_vec], axis=1)

In [495]:
texts[9][0]

['предотвращение',
 'угроза',
 'вирусный',
 'атака',
 'в',
 'автоматизированный',
 'система',
 'введение',
 'за',
 'последний',
 'год',
 'локальный',
 'вычислительный',
 'сеть',
 'лвс',
 'совершить',
 'неоспоримый',
 'технологический',
 'прорыв',
 'стать',
 'неотъемлемый',
 'составлять',
 'производственно-',
 'технологический',
 'база',
 'любой',
 'организация',
 'как',
 'крупный',
 'так',
 'среднее',
 'и',
 'малое',
 'бизнес',
 'не',
 'только',
 'в',
 'россия',
 'но',
 'и',
 'за',
 'рубеж',
 'корпоративный',
 'информационный',
 'система',
 'становиться',
 'сегодня',
 'один',
 'из',
 'главный',
 'инструмент',
 'управление',
 'бизнес',
 'и',
 'фактически',
 'важный',
 'средство',
 'производство',
 'любой',
 'компания',
 'однако',
 'новый',
 'информационный',
 'технология',
 'применять',
 'в',
 'лвс',
 'не',
 'только',
 'значительно',
 'расширить',
 'возможность',
 'организация',
 'но',
 'и',
 'сделать',
 'они',
 'более',
 'уязвимый',
 'для',
 'вероятный',
 'нарушитель',
 'информационный

In [575]:
clusters_count = len(texts)
print(clusters_count)
d_mat = distance_matrix
clusters = texts
clusters = np.array(clusters)
while clusters_count > 2:
    x = np.where(d_mat==np.min(d_mat[np.nonzero(d_mat)]))[0]
    print(x)
    d_mat = update_distance_matrix(x[0], x[1], d_mat)
    clusters[x[0]][0] = np.append(clusters[x[0]][0], clusters[x[1]][0], axis=0)
#     print(clusters.shape)
    clusters = np.delete(clusters, x[1], axis=0)
    clusters_count -= 1

10
[1 2 4 5]
[2 6]
[1 2]
[2 6]
[1 4]
[0 4]
[1 2]
[1 2]


In [573]:
clusters

array([[array(['критерий', 'сопоставимость', 'двуязычный', ..., 'реалия',
               'спортивный', 'действительность'], dtype='<U22')          ],
       [array(['обнаружение', 'аномальный', 'сетевой', ..., 'качество',
               'тестовый', 'воздействие'], dtype='<U23')               ]],
      dtype=object)